In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

from keras import preprocessing
from keras.preprocessing.text import Tokenizer
import numpy as np

2024-06-22 17:10:09.278078: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 17:10:10.679764: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 17:10:14.434897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import resample
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [33]:
df = pd.read_csv('AI_vs_Human.csv')
df

,text,generated
0,Бои у Сопоцкина и Друскеник закончились отступ...,0.0
1,"Министерство народного просвещения, в виду про...",0.0
2,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",0.0
3,Фотограф-корреспондент Daily Mirror рассказыва...,0.0
4,"Лица, приехавшие в Варшаву из Люблина, передаю...",0.0
...,...,...
888281,Tie Face on Mars is really just a big misunder...,0.0
888282,The whole purpose of democracy is to create a ...,0.0
888283,I firmly believe that governments worldwide sh...,1.0
888284,I DFN't agree with this decision because a LFT...,0.0


In [41]:
df.head()

,text,generated
0,Бои у Сопоцкина и Друскеник закончились отступ...,0.0
1,"Министерство народного просвещения, в виду про...",0.0
2,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",0.0
3,Фотограф-корреспондент Daily Mirror рассказыва...,0.0
4,"Лица, приехавшие в Варшаву из Люблина, передаю...",0.0


In [47]:
print(df.info())

print(df.describe())

print(df['text'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887219 entries, 0 to 888285
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   text       887219 non-null  object 
 1   generated  887219 non-null  float64
dtypes: float64(1), object(1)
memory usage: 20.3+ MB
None
           generated
count  887219.000000
mean        0.221312
std         0.415130
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
ООН                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [43]:
print(df.isnull().sum())

text         0
generated    0
dtype: int64


In [46]:
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 0


In [48]:
df['text_length'] = df['text'].apply(len)

print(df['text_length'].describe())

count    887219.000000
mean       1811.777260
std         981.042633
min           1.000000
25%        1121.000000
50%        1615.000000
75%        2281.000000
max       55387.000000
Name: text_length, dtype: float64


In [49]:
df.head()

,text,generated,text_length
0,Бои у Сопоцкина и Друскеник закончились отступ...,0.0,901
1,"Министерство народного просвещения, в виду про...",0.0,346
2,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",0.0,1923
3,Фотограф-корреспондент Daily Mirror рассказыва...,0.0,703
4,"Лица, приехавшие в Варшаву из Люблина, передаю...",0.0,754


In [50]:
q1 = df['text_length'].quantile(0.25)
q3 = df['text_length'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)
outliers = df[(df['text_length'] < lower_bound) | (df['text_length'] > upper_bound)]

df_no_outliers = df[(df['text_length'] > lower_bound) & (df['text_length'] < upper_bound)]

In [53]:
# Separate the dataset into human and AI-generated texts
df_human = df_no_outliers[df_no_outliers['generated'] == 0]
df_ai = df_no_outliers[df_no_outliers['generated'] == 1]

# Undersample the human-generated texts
df_human_undersampled = resample(df_human, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_ai),     # to match minority class
                                 random_state=123) # reproducible results

# Combine the undersampled human-generated texts with AI-generated texts
df_balanced = pd.concat([df_human_undersampled, df_ai])

# Shuffle the dataset to mix the order of rows
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)

In [64]:
df_balanced

,text,generated,text_length,clean_text
0,"Residents in Vauban, Germany have done the unt...",0.0,1690,residents vauban germany done unthinkable give...
1,Индексация пенсий в 2005 году составит в средн...,0.0,263,индексация пенсий году составит среднем рубля ...
2,"Hey, I'm totally stoked to be writing this ess...",1.0,1513,hey im totally stoked writing essay benefits t...
3,"Ливанские военные и полицейские, взятые в плен...",0.0,728,ливанские военные полицейские взятые плен изра...
4,5 июня в 15:00 в Городском парке культуры и от...,1.0,231,июня городском парке культуры отдыха состоится...
...,...,...,...,...
382133,The use ob cell phones in the classroom For ed...,1.0,1566,use ob cell phones classroom educational purpo...
382134,Title: The Benefits of Reducing Automobile Dep...,1.0,3261,title benefits reducing automobile dependence ...
382135,Seeking multiple opinions when making AK impor...,1.0,2689,seeking multiple opinions making ak important ...
382136,"Dear Mr Senator, what's the point in voting if...",0.0,2294,dear mr senator whats point voting vote may ev...


In [54]:
nltk.download('stopwords')
stop_words_ru = set(stopwords.words('russian'))

stop_words_en = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^а-яёa-z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words_ru and word not in stop_words_en])
    return text

df_balanced['clean_text'] = df_balanced['text'].apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [55]:

print(df_balanced['clean_text'].describe())

print(df_balanced.head())

print(df_balanced.info())

print(df_balanced['clean_text'].head())

count     382138
unique    373776
top             
freq         118
Name: clean_text, dtype: object
                                                text  ...                                         clean_text
0  Residents in Vauban, Germany have done the unt...  ...  residents vauban germany done unthinkable give...
1  Индексация пенсий в 2005 году составит в средн...  ...  индексация пенсий году составит среднем рубля ...
2  Hey, I'm totally stoked to be writing this ess...  ...  hey im totally stoked writing essay benefits t...
3  Ливанские военные и полицейские, взятые в плен...  ...  ливанские военные полицейские взятые плен изра...
4  5 июня в 15:00 в Городском парке культуры и от...  ...  июня городском парке культуры отдыха состоится...

[5 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382138 entries, 0 to 382137
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   text         382138 non

In [56]:
X = df_balanced['clean_text']
y = df_balanced['generated']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=100)
X_test_pad = pad_sequences(X_test_seq, maxlen=100)

In [59]:
def load_glove_embeddings(glove_path, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

glove_path = 'glove.6B.100d.txt'  
embedding_dim = 100
embedding_matrix = load_glove_embeddings(glove_path, tokenizer.word_index, embedding_dim)

In [60]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

def create_model_with_glove(embedding_matrix, input_length=100):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0], 
                  output_dim=embedding_matrix.shape[1], 
                  weights=[embedding_matrix], 
                  input_length=input_length, 
                  trainable=False),
        LSTM(units=50, dropout=0.2, recurrent_dropout=0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model_with_glove(embedding_matrix, input_length=100)

2024-06-22 18:04:02.159631: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [61]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          59271600  
                                                                 
 lstm (LSTM)                 (None, 50)                30200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 59,301,851
Trainable params: 30,251
Non-trainable params: 59,271,600
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train_pad, y_train, epochs=8, batch_size=64, validation_split=0.1)
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy}')
print(f'Test Loss: {loss}')

Epoch 1/8
4300/4300 [==============================] - 294s 68ms/step - loss: 0.1184 - accuracy: 0.9520 - val_loss: 0.1151 - val_accuracy: 0.9525
Epoch 2/8
4300/4300 [==============================] - 291s 68ms/step - loss: 0.1151 - accuracy: 0.9527 - val_loss: 0.1134 - val_accuracy: 0.9522
Epoch 3/8
3827/4300 [=========================>....] - ETA: 31s - loss: 0.1117 - accuracy: 0.9533

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc = history_dict['acc']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history_dict['acc']
val_acc = history_dict['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()